# <font color=orange><div align="center">SDP Project - Question 3</div></font>

### <font color=orange><div align="center">15/12/2025</div></font>
### <font color=orange><div align="center">Ouissal BOUTOUATOU - Alae TAOUDI - Mohammed SBAIHI</div></font>


Refer to [Problem Formulation](Question4.md) for rigorous mathematical formulation of the problem

In [29]:
from gurobipy import *

## Instanciation

In [30]:
CANDIDATES = {
    "x": [85, 81, 71, 69, 75, 81, 88],
    "y": [81, 81, 75, 63, 67, 88, 95],
    "z": [74, 89, 74, 81, 68, 84, 79],
    "t": [74, 71, 84, 91, 77, 76, 73],
    "u": [72, 75, 66, 85, 88, 66, 93],
    "v": [71, 73, 63, 92, 76, 79, 93],
    "w": [79, 69, 78, 76, 67, 84, 79],
    "w'": [57, 76, 81, 76, 82, 86, 77],
    "a1": [89, 74, 81, 68, 84, 79, 77],
    "a2": [71, 84, 91, 79, 78, 73.5, 77]
}



WEIGHTS = [8, 7, 7, 6, 6, 5, 6]
COURSES = ["A", "B", "C", "D", "E", "F", "G"]

In [31]:
def pros_and_cons(x,y):
    """Returns the pros(x,y) and cons(x, y) mappings: Course -> Contribution to x > y
    Inputs: x, y students
    N.B: courses are annotated using integers instead of alphabets
    """
    assert x in CANDIDATES, f"{x} isn't in candidates"
    assert y in CANDIDATES, f"{y} isn't in candidates"

    pros = {
        i : (CANDIDATES[x][i] - CANDIDATES[y][i])*WEIGHTS[i]
        for i in range(len(CANDIDATES[x])) 
        if (CANDIDATES[x][i] - CANDIDATES[y][i]) > 0
    }

    cons = {
        i : (CANDIDATES[x][i] - CANDIDATES[y][i])*WEIGHTS[i]
        for i in range(len(CANDIDATES[x])) 
        if (CANDIDATES[x][i] - CANDIDATES[y][i]) < 0
    }
    return pros, cons

## Part 1: Demonstrating that no (1-m) or (m-1) explanation exists for z > t

In [32]:
# Analysis for z > t
PROS_ZT, CONS_ZT = pros_and_cons("z", "t")
print("Pros(z, t):", PROS_ZT)
print("Cons(z, t):", CONS_ZT)
print("\nDetailed contributions:")
for course_idx, contrib in PROS_ZT.items():
    print(f"  {COURSES[course_idx]}: +{contrib}")
for course_idx, contrib in CONS_ZT.items():
    print(f"  {COURSES[course_idx]}: {contrib}")

Pros(z, t): {1: 126, 5: 40, 6: 36}
Cons(z, t): {2: -70, 3: -60, 4: -54}

Detailed contributions:
  B: +126
  F: +40
  G: +36
  C: -70
  D: -60
  E: -54


### Exhaustive verification: Check if any (1-m) trade-offs exist


In [33]:
from itertools import combinations

print("Testing all possible (1-m) trade-offs:\n")
print("=" * 80)

valid_tradeoffs = {}

# For each pros course, check all possible subsets of cons
for p in PROS_ZT:
    pros_course = COURSES[p]
    pros_contrib = PROS_ZT[p]
    
    print(f"\nPros course {pros_course} (+{pros_contrib}):")
    
    valid_for_this_pros = []
    
    # Check all non-empty subsets of cons
    for r in range(1, len(CONS_ZT) + 1):
        for cons_subset in combinations(CONS_ZT.keys(), r):
            cons_contrib = sum(CONS_ZT[c] for c in cons_subset)
            total = pros_contrib + cons_contrib
            
            cons_names = [COURSES[c] for c in cons_subset]
            
            if total > 0:
                cons_list = ', '.join(cons_names)
                print(f"  [OK] With {{{cons_list}}}: {pros_contrib} + ({cons_contrib}) = {total} > 0")
                valid_for_this_pros.append(cons_subset)
            else:
                cons_list = ', '.join(cons_names)
                print(f"  [X] With {{{cons_list}}}: {pros_contrib} + ({cons_contrib}) = {total} ≤ 0")
    
    valid_tradeoffs[p] = valid_for_this_pros

print("\n" + "=" * 80)
print("SUMMARY OF VALID (1-m) TRADE-OFFS:")
print("=" * 80)

for p, tradeoffs in valid_tradeoffs.items():
    pros_course = COURSES[p]
    if tradeoffs:
        print(f"\nPros {pros_course} can form {len(tradeoffs)} valid trade-off(s):")
        for cons_subset in tradeoffs:
            cons_names = [COURSES[c] for c in cons_subset]
            cons_list = ', '.join(cons_names)
            print(f"  - ({pros_course}, {{{cons_list}}})")
    else:
        print(f"\nPros {pros_course}: NO valid trade-offs")

Testing all possible (1-m) trade-offs:


Pros course B (+126):
  [OK] With {C}: 126 + (-70) = 56 > 0
  [OK] With {D}: 126 + (-60) = 66 > 0
  [OK] With {E}: 126 + (-54) = 72 > 0
  [X] With {C, D}: 126 + (-130) = -4 ≤ 0
  [OK] With {C, E}: 126 + (-124) = 2 > 0
  [OK] With {D, E}: 126 + (-114) = 12 > 0
  [X] With {C, D, E}: 126 + (-184) = -58 ≤ 0

Pros course F (+40):
  [X] With {C}: 40 + (-70) = -30 ≤ 0
  [X] With {D}: 40 + (-60) = -20 ≤ 0
  [X] With {E}: 40 + (-54) = -14 ≤ 0
  [X] With {C, D}: 40 + (-130) = -90 ≤ 0
  [X] With {C, E}: 40 + (-124) = -84 ≤ 0
  [X] With {D, E}: 40 + (-114) = -74 ≤ 0
  [X] With {C, D, E}: 40 + (-184) = -144 ≤ 0

Pros course G (+36):
  [X] With {C}: 36 + (-70) = -34 ≤ 0
  [X] With {D}: 36 + (-60) = -24 ≤ 0
  [X] With {E}: 36 + (-54) = -18 ≤ 0
  [X] With {C, D}: 36 + (-130) = -94 ≤ 0
  [X] With {C, E}: 36 + (-124) = -88 ≤ 0
  [X] With {D, E}: 36 + (-114) = -78 ≤ 0
  [X] With {C, D, E}: 36 + (-184) = -148 ≤ 0

SUMMARY OF VALID (1-m) TRADE-OFFS:

Pros B can form

### Exhaustive verification: Check if any (m-1) trade-offs exist


In [34]:
print("Testing all possible (m-1) trade-offs for z > t:\n")
print("=" * 80)

# Dictionary to store valid combinations for each cons course
valid_m1_tradeoffs = {}

# Iterate through each negative contribution (CONS)
# In an (m-1) explanation, each con must be the pivot of a trade-off
for c in CONS_ZT:
    cons_course = COURSES[c]
    cons_contrib = CONS_ZT[c]
    
    print(f"\nCons course {cons_course} ({cons_contrib}):")
    
    valid_for_this_cons = []
    
    # Test every possible combination of pros (from 1 pro up to the total number of pros)
    # This checks if a group of pros can outweigh a single con
    for r in range(1, len(PROS_ZT) + 1):
        for pros_subset in combinations(PROS_ZT.keys(), r):
            pros_contrib = sum(PROS_ZT[p] for p in pros_subset)
            total = pros_contrib + cons_contrib
            
            pros_names = [COURSES[p] for p in pros_subset]
            
            # A trade-off is valid only if the sum of contributions is strictly positive
            if total > 0:
                pros_list = ', '.join(pros_names)
                print(f"  [OK] With {{{pros_list}}}: ({pros_contrib}) + {cons_contrib} = {total} > 0")
                valid_for_this_cons.append(pros_subset)
            else:
                pros_list = ', '.join(pros_names)
                print(f"  [X] With {{{pros_list}}}: ({pros_contrib}) + {cons_contrib} = {total} ≤ 0")
    
    valid_m1_tradeoffs[c] = valid_for_this_cons

print("\n" + "=" * 80)
print("SUMMARY OF VALID (m-1) TRADE-OFFS:")
print("=" * 80)

# Display the summary of which pros can cover which con
for c, tradeoffs in valid_m1_tradeoffs.items():
    cons_course = COURSES[c]
    if tradeoffs:
        print(f"\nCons {cons_course} can be covered by {len(tradeoffs)} valid combinations of pros:")
        for pros_subset in tradeoffs:
            pros_names = [COURSES[p] for p in pros_subset]
            pros_list = ', '.join(pros_names)
            print(f"  - ({{{pros_list}}}, {cons_course})")
    else:
        # If no combination of pros works, no (m-1) explanation can exist for this con
        print(f"\nCons {cons_course}: NO valid trade-offs found with current pros.")

Testing all possible (m-1) trade-offs for z > t:


Cons course C (-70):
  [OK] With {B}: (126) + -70 = 56 > 0
  [X] With {F}: (40) + -70 = -30 ≤ 0
  [X] With {G}: (36) + -70 = -34 ≤ 0
  [OK] With {B, F}: (166) + -70 = 96 > 0
  [OK] With {B, G}: (162) + -70 = 92 > 0
  [OK] With {F, G}: (76) + -70 = 6 > 0
  [OK] With {B, F, G}: (202) + -70 = 132 > 0

Cons course D (-60):
  [OK] With {B}: (126) + -60 = 66 > 0
  [X] With {F}: (40) + -60 = -20 ≤ 0
  [X] With {G}: (36) + -60 = -24 ≤ 0
  [OK] With {B, F}: (166) + -60 = 106 > 0
  [OK] With {B, G}: (162) + -60 = 102 > 0
  [OK] With {F, G}: (76) + -60 = 16 > 0
  [OK] With {B, F, G}: (202) + -60 = 142 > 0

Cons course E (-54):
  [OK] With {B}: (126) + -54 = 72 > 0
  [X] With {F}: (40) + -54 = -14 ≤ 0
  [X] With {G}: (36) + -54 = -18 ≤ 0
  [OK] With {B, F}: (166) + -54 = 112 > 0
  [OK] With {B, G}: (162) + -54 = 108 > 0
  [OK] With {F, G}: (76) + -54 = 22 > 0
  [OK] With {B, F, G}: (202) + -54 = 148 > 0

SUMMARY OF VALID (m-1) TRADE-OFFS:

Cons C 

Aucune combinaison n'utilise des pros différents pour les trois cons à la fois!

## Part 2: Linear Program for hybride explanation

### Variables

In [35]:
PROS, CONS = pros_and_cons("z", "t")
print("The Pros Set:", PROS)
print("The Cons Set:", CONS)

The Pros Set: {1: 126, 5: 40, 6: 36}
The Cons Set: {2: -70, 3: -60, 4: -54}


In [36]:
m = Model("Hybrid_Explanations")

In [37]:
# 1. Assignment variables
VarAssignPro = {(p, c) : m.addVar(vtype = GRB.BINARY, name=f'atp_{p}_{c}')
                    for p in PROS for c in CONS}

VarAssignCon = {(p, c) : m.addVar(vtype = GRB.BINARY, name=f'atc_{p}_{c}')
                    for p in PROS for c in CONS}

# 2. Pivot variables
VarPivotPro = {p : m.addVar(vtype = GRB.BINARY, name=f'piv_p_{p}')
                    for p in PROS}

VarPivotCon = {c : m.addVar(vtype = GRB.BINARY, name=f'piv_c_{c}')
                    for c in CONS}

### Constraints

1. Every con course must be covered exactly onceEach con is either an (m-1) pivot or assigned to a (1-m) pivot.$$\forall j \in [1, C], pivot\_con_j + \sum_{i=1}^{P} assign\_to\_pro_{i, j} = 1$$

In [38]:
CON_COVERAGE = {
    c: m.addConstr(
        VarPivotCon[c] + quicksum([VarAssignPro[(p, c)] for p in PROS]) == 1
    ) for c in CONS
}

Every pro course can be used at most onceA pro is either a (1-m) pivot or assigned to at most one (m-1) pivot.$$\forall i \in [1, P], pivot\_pro_i + \sum_{j=1}^{C} assign\_to\_con_{i, j} \leq 1$$

In [39]:
PRO_USAGE = {
    p: m.addConstr(
        VarPivotPro[p] + quicksum([VarAssignCon[(p, c)] for c in CONS]) <= 1
    ) for p in PROS
}

Linking Constraints Assignments can only exist if the respective pivots are active.

In [40]:
LINK_ATP = {
    (p, c): m.addConstr(VarAssignPro[(p, c)] <= VarPivotPro[p])
    for p in PROS for c in CONS
}

LINK_ATC = {
    (p, c): m.addConstr(VarAssignCon[(p, c)] <= VarPivotCon[c])
    for p in PROS for c in CONS
}

Trade-off Validity (Corrected with Big-M)We use a large constant $M$ to ensure the constraint is only active when the course is a pivot.$$\forall i \in [1, P], p_i + \sum_{j=1}^{C} assign\_to\_pro_{i, j} \cdot c_j \geq pivot\_pro_i \cdot \epsilon - (1 - pivot\_pro_i) \cdot M$$

In [41]:
EPSILON = 0.01
BIG_M = 1000  # Large constant to relax constraints

VALIDITY_1M = {
    p: m.addConstr(
        PROS[p] + quicksum([VarAssignPro[(p, c)] * CONS[c] for c in CONS]) 
        >= VarPivotPro[p] * EPSILON - (1 - VarPivotPro[p]) * BIG_M
    ) for p in PROS
}

VALIDITY_M1 = {
    c: m.addConstr(
        CONS[c] + quicksum([VarAssignCon[(p, c)] * PROS[p] for p in PROS]) 
        >= VarPivotCon[c] * EPSILON - (1 - VarPivotCon[c]) * BIG_M
    ) for c in CONS
}

### Objective Function

Minimize the total number of arguments (pivots):$$\min \sum_{i=1}^{P} pivot\_pro_i + \sum_{j=1}^{C} pivot\_con_j$$

In [42]:
obj = quicksum(VarPivotPro[p] for p in PROS) + quicksum(VarPivotCon[c] for c in CONS)
m.setObjective(obj, GRB.MINIMIZE)


### Solver  

In [43]:
m.optimize()

Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-8350U CPU @ 1.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 30 rows, 24 columns and 84 nonzeros (Min)
Model fingerprint: 0x017c4a64
Model has 6 linear objective coefficients
Variable types: 0 continuous, 24 integer (24 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 2.0000000
Presolve removed 30 rows and 24 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+00, best bound 2.000000000000e+00, gap 0

## Results and Interpretation

### Checking the hybrid explanation for candidates z and t 

In [44]:
if m.status == GRB.OPTIMAL:
    print("=" * 80)
    print("HYBRID OPTIMAL SOLUTION FOUND!")
    print("=" * 80)
    print(f"\nMinimum number of arguments (pivots): {int(m.objVal)}")
    
    # --- 1. Display (1-m) Trade-offs ---
    print("\n" + "=" * 80)
    print("EXPLANATION TYPE (1-m):")
    print("=" * 80)
    for p in PROS:
        if VarPivotPro[p].X > 0.5:
            # Find all cons associated with this pro pivot
            associated_cons = [c for c in CONS if VarAssignPro[(p, c)].X > 0.5]
            
            total_contrib = PROS[p] + sum(CONS[c] for c in associated_cons)
            pros_name = COURSES[p]
            cons_names = [COURSES[c] for c in associated_cons]
            
            print(f"\nTrade-off: ({pros_name}, {{{', '.join(cons_names)}}})")
            print(f"  - Pros contribution [{pros_name}]: {PROS[p]:+d}")
            for c in associated_cons:
                print(f"  - Cons contribution [{COURSES[c]}]: {CONS[c]:+d}")
            print(f"  - Total contribution: {total_contrib:+.2f}")

    # --- 2. Display (m-1) Trade-offs ---
    print("\n" + "=" * 80)
    print("EXPLANATION TYPE (m-1):")
    print("=" * 80)
    for c in CONS:
        if VarPivotCon[c].X > 0.5:
            # Find all pros associated with this con pivot
            associated_pros = [p for p in PROS if VarAssignCon[(p, c)].X > 0.5]
            
            total_contrib = CONS[c] + sum(PROS[p] for p in associated_pros)
            cons_name = COURSES[c]
            pros_names = [COURSES[p] for p in associated_pros]
            
            print(f"\nTrade-off: ({{{', '.join(pros_names)}}}, {cons_name})")
            for p in associated_pros:
                print(f"  - Pros contribution [{COURSES[p]}]: {PROS[p]:+d}")
            print(f"  - Cons contribution [{cons_name}]: {CONS[c]:+d}")
            print(f"  - Total contribution: {total_contrib:+.2f}")

    # --- 3. Final Verification ---
    print("\n" + "=" * 80)
    print("VERIFICATION:")
    print("=" * 80)
    
    covered_cons = set()
    # Check cons covered by 1-m
    for (p, c) in VarAssignPro:
        if VarAssignPro[(p, c)].X > 0.5:
            covered_cons.add(c)
    # Check cons covered by m-1 (pivots)
    for c in CONS:
        if VarPivotCon[c].X > 0.5:
            covered_cons.add(c)
            
    cons_to_cover = set(CONS.keys())
    print(f"All cons covered: {covered_cons == cons_to_cover}")
    print(f"Cons covered: {[COURSES[c] for c in covered_cons]}")

elif m.status == GRB.INFEASIBLE:
    print("=" * 80)
    print("MODEL IS INFEASIBLE!")
    print("=" * 80)
    print("\nNo hybrid explanation exists for this comparison.")
    print("A certificate of non-existence is established.") 

HYBRID OPTIMAL SOLUTION FOUND!

Minimum number of arguments (pivots): 2

EXPLANATION TYPE (1-m):

Trade-off: (B, {D, E})
  - Pros contribution [B]: +126
  - Cons contribution [D]: -60
  - Cons contribution [E]: -54
  - Total contribution: +12.00

EXPLANATION TYPE (m-1):

Trade-off: ({F, G}, C)
  - Pros contribution [F]: +40
  - Pros contribution [G]: +36
  - Cons contribution [C]: -70
  - Total contribution: +6.00

VERIFICATION:
All cons covered: True
Cons covered: ['C', 'D', 'E']


### Checking this model for the new candidates a1 and a2

In [17]:
# Analysis for z > t
PROS_, CONS_ = pros_and_cons("a1", "a2")
print("Pros(a1, a2):", PROS_)
print("Cons(a1, a2):", CONS_)
print("\nDetailed contributions:")
for course_idx, contrib in PROS_.items():
    print(f"  {COURSES[course_idx]}: +{contrib}")
for course_idx, contrib in CONS_.items():
    print(f"  {COURSES[course_idx]}: {contrib}")

Pros(a1, a2): {0: 144, 4: 36, 5: 27.5}
Cons(a1, a2): {1: -70, 2: -70, 3: -66}

Detailed contributions:
  A: +144
  E: +36
  F: +27.5
  B: -70
  C: -70
  D: -66


In [19]:
PROS = PROS_
CONS = CONS_

# Re-run  Model Initialization
m = Model("Question_4_Final_Check for a1 and a2")

ReRun the cells from Varibales to Objective function ! 

In [27]:
m.optimize()

Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-8350U CPU @ 1.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 30 rows, 24 columns and 84 nonzeros (Min)
Model fingerprint: 0xfd13cfc7
Model has 6 linear objective coefficients
Variable types: 0 continuous, 24 integer (24 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 28 rows and 24 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


The model is infeasible.

* Even with the most flexible model (Hybrid), the solver returns a status of INFEASIBLE. This acts as a formal certificate of non-existence. It proves that there is no possible way to partition the pros and cons of candidate $a_1$ into a set of disjoint (1-m) or (m-1) arguments that cover all of $a_2$'s advantages2.

* Although $a_1$ has a higher weighted average than $a_2$, this superiority cannot be explained through simple 'one-to-many' or 'many-to-one' arguments. The advantage $a_1$ holds in Anatomy is almost entirely exhausted by covering just two of your strengths (Biology and Surgery). The remaining advantages ($a_1$'s scores in Epidemiology and Forensic) are simply not strong enough to overcome your significant lead in Diagnostic. Their overall lead is the result of a complex global balance (trade-off $m$-to-$n$) that exceeds the simplified logic of the requested explanation.

